$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ 20165549\ \ JANG\ \ JAE\ \ YONG$$

Build a binary classifier for human versus horse based on logistic regression using the dataset that consists of human and horse images

The dataset consists of human images and horse images for the training and the validation

The classifier should be trained using the training set

The classifier should be tested using the validation set

In [18]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os

import timeit

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),
    # the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = 'horse-or-human/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)  

validation_data_path = 'horse-or-human/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=1)  

NUM_EPOCH = range(1) # my code

#
# Image size and number of data
#
image_size = 10000
train_count = 1027
validation_count = 256

train_label = np.empty(train_count, dtype=float)
train_data = np.ones((train_count, image_size + 1), dtype=float)

validation_label = np.empty(validation_count, dtype=float)
validation_data = np.ones((validation_count, image_size + 1), dtype=float)

for epoch in (NUM_EPOCH):
    # load training images of the batch size for every iteration
    for i, data in enumerate(trainloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)
        
        train_label[i] = int(labels)
        train_tmp = np.asfarray(inputs)
        train_data[i, :10000] = train_tmp[0, 0, :, :].reshape(10000)
        
    # load validation images of the batch size for every iteration
    for i, data in enumerate(valloader):
        
        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)
        
        validation_label[i] = int(labels)
        validation_tmp = np.asfarray(inputs)
        validation_data[i, :10000] = validation_tmp[0, 0, :, :].reshape(10000)

In [19]:
np.random.seed(1)

learningRate = 10**(-5)
delta = 10**(-9)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def loss_function(t, y):
    return -np.sum(t*np.log(y+delta) + (1-t)*np.log(1-y+delta)) 

iteration = 5000

d = np.empty(train_count, dtype=float)
w = np.empty(image_size + 1, dtype=float)

loss_arr = np.zeros(iteration, dtype=float)
t_loss_arr = np.zeros(iteration, dtype=float)
loss_value_arr = np.zeros(iteration, dtype=float)
t_loss_value_arr = np.zeros(iteration, dtype=float)
iteration_time = np.zeros(iteration, dtype=float)

for i in range(image_size + 1):
    w[i] = np.random.rand(1)

for iter in range(iteration):
    
    start = timeit.default_timer()

    
    for i in range(train_count):
        d[i] = sigmoid(w[:].dot(train_data[i, :])) - train_label[i]

    for i in range(image_size + 1):
        w[i] = w[i] - learningRate * np.sum(d[:].dot(train_data[:, i]))
   
    loss_value = 0
    t_loss_value = 0
    training_loss = 0
    testing_loss = 0
    
    for i in range(train_count):
        loss_value += loss_function(train_label[i], sigmoid(w[:].dot(train_data[i, :])))

        if train_label[i] == 1:
            if sigmoid(w[:].dot(train_data[i, :])) <= 0.5: # loss count
                training_loss += 1

        else:
            if sigmoid(w[:].dot(train_data[i, :])) > 0.5: # loss count
                training_loss += 1

    for i in range(validation_count):
        t_loss_value += loss_function(validation_label[i], sigmoid(w[:].dot(validation_data[i, :])))
        if validation_label[i] == 1:
            if sigmoid(w[:].dot(validation_data[i, :])) <= 0.5: # loss count
                testing_loss += 1

        else:
            if sigmoid(w[:].dot(validation_data[i, :])) > 0.5: # loss count
                testing_loss += 1

    if iter % 10 == 0:
        print("iter =", iter)
        print("train loss count =", training_loss)
        print("train loss value =", loss_value)
        print("validation loss count =", testing_loss)
        print("validation loss value =", t_loss_value)
        
    
    loss_value_arr[iter] = loss_value
    t_loss_value_arr[iter] = t_loss_value
    loss_arr[iter] = training_loss
    t_loss_arr[iter] = testing_loss
    
    stop = timeit.default_timer()
    
    iteration_time[iter] = stop - start
    
    if iter % 10 == 0:
        print(iter, 'iteration time =', iteration_time[iter])
    

iter = 0
train loss count = 500
train loss value = 10361.63291794603
validation loss count = 128
validation loss value = 2652.5780270011383
0 iteration time = 0.2175913999999466
iter = 10
train loss count = 500
train loss value = 10361.63291794603
validation loss count = 128
validation loss value = 2652.5780270011383
10 iteration time = 0.21390270000028977
iter = 20
train loss count = 500
train loss value = 10361.63291794603
validation loss count = 128
validation loss value = 2652.5780270011383
20 iteration time = 0.18531009999969683
iter = 30
train loss count = 500
train loss value = 10361.63291794603
validation loss count = 128
validation loss value = 2652.5780270011383
30 iteration time = 0.19031569999970088
iter = 40
train loss count = 500
train loss value = 10361.63291794603
validation loss count = 128
validation loss value = 2652.5780270011383
40 iteration time = 0.22262190000037663
iter = 50
train loss count = 500
train loss value = 10361.63291794603
validation loss count = 128


iter = 450
train loss count = 280
train loss value = 1264.3283528023342
validation loss count = 49
validation loss value = 198.4033844464263
450 iteration time = 0.18602779999946506
iter = 460
train loss count = 281
train loss value = 1216.3403745073033
validation loss count = 51
validation loss value = 185.6114438576825
460 iteration time = 0.1780404000000999
iter = 470
train loss count = 282
train loss value = 1172.4453711717258
validation loss count = 47
validation loss value = 175.94297811224044
470 iteration time = 0.18077350000021397
iter = 480
train loss count = 277
train loss value = 1132.2578922930825
validation loss count = 45
validation loss value = 168.70772040743282
480 iteration time = 0.17257940000035887
iter = 490
train loss count = 276
train loss value = 1095.495302685649
validation loss count = 44
validation loss value = 163.39499732128962
490 iteration time = 0.20826349999970262
iter = 500
train loss count = 276
train loss value = 1061.9238629511465
validation loss c

train loss count = 202
train loss value = 568.4469007798484
validation loss count = 45
validation loss value = 190.36908824519875
900 iteration time = 0.2024531000006391
iter = 910
train loss count = 199
train loss value = 562.9299343898119
validation loss count = 45
validation loss value = 191.23591704192654
910 iteration time = 0.21923379999952886
iter = 920
train loss count = 196
train loss value = 557.5479692984541
validation loss count = 45
validation loss value = 192.08498799508368
920 iteration time = 0.23587529999986145
iter = 930
train loss count = 196
train loss value = 552.2957071209894
validation loss count = 46
validation loss value = 192.9158945992045
930 iteration time = 0.22058019999985845
iter = 940
train loss count = 192
train loss value = 547.1680994400358
validation loss count = 47
validation loss value = 193.72830176130554
940 iteration time = 0.18235379999987344
iter = 950
train loss count = 190
train loss value = 542.1603293137138
validation loss count = 47
valid

1340 iteration time = 0.21063820000017586
iter = 1350
train loss count = 139
train loss value = 403.45487713477394
validation loss count = 50
validation loss value = 213.0861967650444
1350 iteration time = 0.19304710000051273
iter = 1360
train loss count = 138
train loss value = 400.94127626044497
validation loss count = 50
validation loss value = 213.32394676737954
1360 iteration time = 0.20403150000038295
iter = 1370
train loss count = 138
train loss value = 398.45706197558883
validation loss count = 51
validation loss value = 213.5556087567517
1370 iteration time = 0.2004739999993035
iter = 1380
train loss count = 138
train loss value = 396.00155079071936
validation loss count = 51
validation loss value = 213.78145780160457
1380 iteration time = 0.20622309999998834
iter = 1390
train loss count = 138
train loss value = 393.57408917545075
validation loss count = 51
validation loss value = 214.00176028979484
1390 iteration time = 0.20485590000043885
iter = 1400
train loss count = 138
t

train loss value = 314.3585609699031
validation loss count = 51
validation loss value = 220.39931404590942
1790 iteration time = 0.2234902999998667
iter = 1800
train loss count = 122
train loss value = 312.73514883806405
validation loss count = 51
validation loss value = 220.53306707060378
1800 iteration time = 0.2208913999993456
iter = 1810
train loss count = 122
train loss value = 311.1259327492919
validation loss count = 51
validation loss value = 220.66646669917856
1810 iteration time = 0.2012382000002617
iter = 1820
train loss count = 122
train loss value = 309.53073862014753
validation loss count = 51
validation loss value = 220.79953901070036
1820 iteration time = 0.1936759999998685
iter = 1830
train loss count = 122
train loss value = 307.94939603767193
validation loss count = 51
validation loss value = 220.93230753541877
1830 iteration time = 0.1965924999994968
iter = 1840
train loss count = 122
train loss value = 306.38173815347767
validation loss count = 51
validation loss v

...

train loss value = 179.20027261658453
validation loss count = 60
validation loss value = 235.8871766070348
3130 iteration time = 0.2044824000004155
iter = 3140
train loss count = 68
train loss value = 178.58382819844633
validation loss count = 60
validation loss value = 235.983341484723
3140 iteration time = 0.21375020000050426
iter = 3150
train loss count = 67
train loss value = 177.970936810068
validation loss count = 60
validation loss value = 236.07933388032313
3150 iteration time = 0.16841790000034962
iter = 3160
train loss count = 67
train loss value = 177.36156970545693
validation loss count = 60
validation loss value = 236.1751586074502
3160 iteration time = 0.17912350000005972
iter = 3170
train loss count = 65
train loss value = 176.75569851372794
validation loss count = 59
validation loss value = 236.27082047139876
3170 iteration time = 0.1770939999996699
iter = 3180
train loss count = 64
train loss value = 176.15329523108775
validation loss count = 59
validation loss value =

3570 iteration time = 0.18871330000001763
iter = 3580
train loss count = 51
train loss value = 154.61692086213745
validation loss count = 62
validation loss value = 240.10479697833648
3580 iteration time = 0.19860960000005434
iter = 3590
train loss count = 51
train loss value = 154.13617964114025
validation loss count = 62
validation loss value = 240.197115044826
3590 iteration time = 0.1717348000001948
iter = 3600
train loss count = 51
train loss value = 153.65797398418098
validation loss count = 62
validation loss value = 240.28940212881827
3600 iteration time = 0.17295059999923978
iter = 3610
train loss count = 51
train loss value = 153.18228592911393
validation loss count = 62
validation loss value = 240.38165851102977
3610 iteration time = 0.17908940000052098
iter = 3620
train loss count = 51
train loss value = 152.70909766416912
validation loss count = 62
validation loss value = 240.4738843404757
3620 iteration time = 0.18655570000009902
iter = 3630
train loss count = 51
train lo

validation loss value = 244.12759079427224
4020 iteration time = 0.17398570000023028
iter = 4030
train loss count = 47
train loss value = 135.2600877571976
validation loss count = 61
validation loss value = 244.21761070804905
4030 iteration time = 0.19124560000000201
iter = 4040
train loss count = 47
train loss value = 134.8776592603214
validation loss count = 61
validation loss value = 244.3075410022126
4040 iteration time = 0.20491190000029746
iter = 4050
train loss count = 47
train loss value = 134.4970944753116
validation loss count = 61
validation loss value = 244.3973799189169
4050 iteration time = 0.17008249999980762
iter = 4060
train loss count = 47
train loss value = 134.11838137632083
validation loss count = 61
validation loss value = 244.48712572105424
4060 iteration time = 0.2099262000001545
iter = 4070
train loss count = 47
train loss value = 133.74150804719864
validation loss count = 61
validation loss value = 244.57677669303675
4070 iteration time = 0.1779960999992909
it

train loss value = 120.05015867886999
validation loss count = 62
validation loss value = 248.06803305869207
4470 iteration time = 0.20256609999978537
iter = 4480
train loss count = 31
train loss value = 119.73958951293498
validation loss count = 62
validation loss value = 248.15257444399776
4480 iteration time = 0.16618539999944915
iter = 4490
train loss count = 31
train loss value = 119.4304374675424
validation loss count = 62
validation loss value = 248.2369660141265
4490 iteration time = 0.16896009999982198
iter = 4500
train loss count = 31
train loss value = 119.12269388701466
validation loss count = 62
validation loss value = 248.3212067052146
4500 iteration time = 0.17988450000029843
iter = 4510
train loss count = 31
train loss value = 118.81635016633217
validation loss count = 62
validation loss value = 248.40529545958222
4510 iteration time = 0.19683179999992717
iter = 4520
train loss count = 31
train loss value = 118.51139775061728
validation loss count = 62
validation loss va

4910 iteration time = 0.1903984999999011
iter = 4920
train loss count = 24
train loss value = 107.36185109780112
validation loss count = 62
validation loss value = 251.71022201268426
4920 iteration time = 0.18576050000046962
iter = 4930
train loss count = 24
train loss value = 107.1071622013121
validation loss count = 62
validation loss value = 251.7870989139818
4930 iteration time = 0.2041622999995525
iter = 4940
train loss count = 24
train loss value = 106.85354787981831
validation loss count = 62
validation loss value = 251.8637887176544
4940 iteration time = 0.21046240000032412
iter = 4950
train loss count = 24
train loss value = 106.60100158809848
validation loss count = 62
validation loss value = 251.94029097568912
4950 iteration time = 0.19586620000063704
iter = 4960
train loss count = 24
train loss value = 106.34951682674236
validation loss count = 62
validation loss value = 252.0166052614192
4960 iteration time = 0.21065960000032646
iter = 4970
train loss count = 24
train loss